In [1]:
import json
from tqdm.auto import tqdm

# with open('../data/final_preprocessed_data/js_baseline_test_data.json') as f:
#     data = json.load(f)

with open('../data/vdo_filtered/test_data.json') as f:
    prompt_data = json.load(f)

In [2]:
import os
os.environ['HF_HOME'] = './'

In [3]:
from transformers import pipeline
pipe = pipeline("text2text-generation", model="JetBrains-Research/cmg-race-without-history", device=0)

In [4]:
pipe(prompt_data[0]['diff'])[0]['generated_text']

C:\Users\tyfann\anaconda3\envs\llm4commit\lib\site-packages\transformers\generation\utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'add setTimeouts to focus to first node and width'

In [4]:
diffs = []
generated_commit_messages = []

for commit in prompt_data:
    diff = commit['diff']
    diffs.append(diff)

for diff in tqdm(diffs, total=len(diffs), desc='Generating commit messages'):
    prompt = f"""
    The following is a diff which describes the code changes in a commit, Your task is to write a short commit message accordingly.
    {diff}
    According to the diff, the commit message should be:
    """
    generated_commit_messages.append(pipe(prompt)[0]['generated_text'])

Generating commit messages:   0%|          | 0/1992 [00:00<?, ?it/s]

C:\Users\tyfann\anaconda3\envs\llm4commit\lib\site-packages\transformers\generation\utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (569 > 512). Running this sequence through the model will result in indexing errors
--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\tyfann\anaconda3\envs\llm4commit\lib\logging\__init__.py", line 1100, in emit
    msg = self.format(record)
  File "C:\Users\tyfann\anaconda3\envs\llm4commit\lib\logging\__init__.py", line 943, in format
    return fmt.format(record)
  File "C:\Users\tyfann\anaconda3\envs\llm4commit\lib\logging\__init__.py", line 678, in format
    record.message = record.getMessage()
  File "C:\Users\tyfann\anaconda3\envs\llm4commit\lib\logging\__init__.

In [5]:
for item, msg in zip(prompt_data, generated_commit_messages):
    item['race'] = msg

output_file = '../data/vdo_filtered/generation/test_race_v0.json'
with open(output_file, 'w', encoding='UTF-8') as f:
    json.dump(prompt_data, f, ensure_ascii=False, indent=4)

# save msg to a file
with open("../data/vdo_filtered/generation/test_race_v0.txt", 'w', encoding='UTF-8') as file:
    for item in prompt_data:
        file.write(item['race'].replace('\n', '\\n').replace('\r', '\\r') + '\n')

In [11]:
from langchain import hub

prompt = hub.pull("tyfann/llm4commit-rag")
generated_commit_messages = []
for data in tqdm(prompt_data, desc='Generating commit messages'):
    messages = prompt.invoke(
        {"context": data['sim_diff'], "msg": data['sim_msg'], "diff": data['org_diff']}
    ).to_messages()
    generated_commit_messages.append(pipe(messages[0].content, max_new_tokens=1000)[0]['generated_text'])

In [14]:
with open('../data/final_preprocessed_data/js_baseline_test_data.json') as f:
    data = json.load(f)

for item, msg in zip(data[:1000], generated_commit_messages):
    item['race'] = item['msg']

output_file = '../data/final_preprocessed_data/js_baseline/js_baseline_rag_race.json'
with open(output_file, 'w', encoding='UTF-8') as f:
    json.dump(data[:1000], f, ensure_ascii=False, indent=4)